In [14]:
# imports
import pandas as pd
import pickle
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
from openai.embeddings_utils import (
    get_embedding,
    distances_from_embeddings,
    tsne_components_from_embeddings,
    chart_from_components,
    indices_of_nearest_neighbors_from_distances,
)

# constants
EMBEDDING_MODEL = "text-embedding-ada-002"

In [2]:
# load data (full dataset available at http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)
df = pd.read_csv("../app/data/SpotGenTrack/filtered_track_df.csv")

# print dataframe
n_examples = 5
df.head(n_examples)

,artists_id,acousticness,danceability,disc_number,duration_ms,energy,href,id,instrumentalness,key,...,speechiness,tempo,time_signature,track_href,uri,valence,release_date,artists_name,genres,release_year
0,1RyvyyTE3xzB2ZywiAwp0i,0.01510,0.888,1.0,161758.0,0.665,https://api.spotify.com/v1/tracks/6QFzUXTIZXOL...,6QFzUXTIZXOLesQcgmGOsR,0.000000,5.0,...,0.0930,78.021,4.0,https://api.spotify.com/v1/tracks/6QFzUXTIZXOL...,6QFzUXTIZXOLesQcgmGOsR,0.5680,2019-01-18,Future,"['atl hip hop', 'pop', 'pop rap', 'rap', 'sout...",2019
1,0yN7xI1blow9nYIK0R8nM7,0.00129,0.501,1.0,209133.0,0.964,https://api.spotify.com/v1/tracks/5eaTDXlQLlfN...,5eaTDXlQLlfNLfJALhxRyj,0.003930,7.0,...,0.0793,75.038,4.0,https://api.spotify.com/v1/tracks/5eaTDXlQLlfN...,5eaTDXlQLlfNLfJALhxRyj,0.4860,2019-01-11,Buckcherry,"['alternative metal', 'glam metal', 'hard rock...",2019
2,4SqTiwOEdYrNayaGMkc7ia,0.01710,0.722,1.0,234894.0,0.781,https://api.spotify.com/v1/tracks/0rN1Jv3hGnYZ...,0rN1Jv3hGnYZ2zmAXbpnkc,0.000000,8.0,...,0.0321,137.907,4.0,https://api.spotify.com/v1/tracks/0rN1Jv3hGnYZ...,0rN1Jv3hGnYZ2zmAXbpnkc,0.6440,2019-02-15,LÉON,"['dance pop', 'indie cafe pop', 'indie poptimi...",2019
3,5JMLG56F1X5mFmWNmS0iAp,0.92600,0.616,1.0,210887.0,0.210,https://api.spotify.com/v1/tracks/5hNR2TXslTho...,5hNR2TXslThoEgqqdDlt8F,0.000000,7.0,...,0.0371,70.478,4.0,https://api.spotify.com/v1/tracks/5hNR2TXslTho...,5hNR2TXslThoEgqqdDlt8F,0.4470,2019-03-13,Chelsea Cutler,"['dance pop', 'electropop', 'indie poptimism',...",2019
4,2oX42qP5ineK3hrhBECLmj,0.08750,0.641,1.0,196188.0,0.537,https://api.spotify.com/v1/tracks/31Y7dmcZFeJs...,31Y7dmcZFeJsMKXtJNeQya,0.000009,0.0,...,0.0282,112.864,4.0,https://api.spotify.com/v1/tracks/31Y7dmcZFeJs...,31Y7dmcZFeJsMKXtJNeQya,0.0681,2019-02-13,Andy Grammer,"['neo mellow', 'pop', 'pop rap', 'pop rock', '...",2019


## Build cache to save embeddings

In [3]:
# establish a cache of embeddings to avoid recomputing
# cache is a dict of tuples (text, model) -> embedding, saved as a pickle file

# set path to embedding cache
embedding_cache_path = "../app/data/recommendations_embeddings_cache.pkl"

# load the cache if it exists, and save a copy to disk
try:
    embedding_cache = pd.read_pickle(embedding_cache_path)
except FileNotFoundError:
    embedding_cache = {}
with open(embedding_cache_path, "wb") as embedding_cache_file:
    pickle.dump(embedding_cache, embedding_cache_file)

# define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    embedding_cache=embedding_cache
) -> list:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache.keys():
        embedding_cache[(string, model)] = get_embedding(string, model)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

In [4]:
import pandas as pd
import pickle

# Set path to embedding cache
embedding_cache_path = "../app/data/recommendations_embeddings_cache.pkl"

# Load the cache if it exists, and save a copy to disk
try:
    with open(embedding_cache_path, "rb") as embedding_cache_file:
        embedding_cache = pickle.load(embedding_cache_file)
except FileNotFoundError:
    embedding_cache = {}
    with open(embedding_cache_path, "wb") as embedding_cache_file:
        pickle.dump(embedding_cache, embedding_cache_file)

# Define a function to retrieve embeddings from the cache if present, and otherwise request via the API
def embedding_from_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    embedding_cache=embedding_cache
) -> list:
    """Return embedding of given string, using a cache to avoid recomputing."""
    if (string, model) not in embedding_cache:
        embedding_cache[(string, model)] = get_embedding(string, model)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[(string, model)]

In [17]:
# as an example, take the first description from the dataset
#example_string = df["lyrics"].values[0]

example_string = "iloveyou"
print(f"\nExample string: {example_string}")

# print the first 10 dimensions of the embedding
example_embedding = embedding_from_string(example_string)
print(f"\nExample embedding: {example_embedding[:10]}...")


Example string: iloveyou


KeyError: 'lyrics'